In [0]:
import requests
import json
import pyspark.sql.functions as F
from pyspark.sql.types import *
import pyspark
from pyspark.sql.functions import sum,avg,max,min,mean,count

In [0]:
import pandas as pd
import numpy as np
import datetime as dt
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data] Package vader_lexicon is already up-to-date!

In [0]:
spark.conf.set(
  "")
spark.conf.set(
  "")
df = spark.read.csv("wasbs://test@sentimentdatacs4225.blob.core.windows.net/feb-1.csv")

In [0]:
  sid = SentimentIntensityAnalyzer()
  def get_sentiment(sentence):
    if(sentence == None):
      return 0
    ss = sid.polarity_scores(sentence)
    return ss['compound']

In [0]:
udfsomefunc = F.udf(get_sentiment, StringType())
df3 = df.withColumn("sentiment",udfsomefunc(df._c3))
df3.write.format("csv").save("wasbs://test@sentimentdatacs4225.blob.core.windows.net/feb-sentiment-NLTK" + "1" + ".csv")

In [0]:
df3 = spark.read.csv("wasbs://test@sentimentdatacs4225.blob.core.windows.net/feb-sentiment-NLTK1.csv")
df3 = df3.withColumnRenamed("_c0","Date").withColumnRenamed("_c1","Language").withColumnRenamed("_c2","Country").withColumnRenamed("_c3","tweet").withColumnRenamed("_c4","sentimentScore")
df3.show()

+----------+--------+--------------+--------------------+--------------+
 Date|Language| Country| tweet|sentimentScore|
+----------+--------+--------------+--------------------+--------------+
2020-02-01| en| India|mask avoid coron...| -0.296|
2020-02-01| en| Philippines|stan right baobei...| 0.0|
2020-02-01| en| India|unbelevable ameri...| 0.0|
2020-02-01| en| France|airfrance suspend...| 0.0|
2020-02-01| en| India|called chinese st...| 0.3612|
2020-02-01| en|United Kingdom|scientists making...| 0.4215|
2020-02-01| en| India|misinformation co...| 0.0258|
2020-02-01| en|United Kingdom|father can’t touc...| -0.4445|
2020-02-01| en| Philippines|iran bans flights...| 0.0|
2020-02-01| en| India|travel ban austra...| -0.5574|
2020-02-01| en| Canada|russia issues new...| 0.2263|
2020-02-01| en| Canada|major transportat...| 0.0258|
2020-02-01| en| Sweden|there's good way ...| -0.7964|
2020-02-01| en| Malaysia|pray unsung heroe...| 0.5423|
2020-02-01| en| India|student return ch...| 0.4588|
2020-02-01| en| India|christian girl re...| 0.5574|
2020-02-01| en| Canada|“evidence documen...| 0.0|
2020-02-01| en| India|indian home 👍👏...| 0.0|
2020-02-01| en| India|imran khan willin...| -0.7579|
2020-02-01| en| Hong Kong|denies entry chin...| -0.4215|
+----------+--------+--------------+--------------------+--------------+
only showing top 20 rows

In [0]:
df4=df3.groupby("Date","Country").agg(avg("sentimentScore").alias("sentimentScore_avg"))
df4.show()

+----------+--------------+--------------------+
 Date| Country| sentimentScore_avg|
+----------+--------------+--------------------+
2020-02-01| Thailand|0.008028030303030311|
2020-02-01| Philippines|-0.02492216705607475|
2020-02-01| Germany| -0.0745345226615236|
2020-02-01| China|0.011914330462020355|
2020-02-01| Hong Kong|-0.19864234714266674|
2020-02-01| Malaysia|-0.14656846726190553|
2020-02-01|United Kingdom|-0.10030697115384611|
2020-02-01| India|-0.02818806267548...|
2020-02-01| Canada|-0.07343756456098528|
2020-02-01| France|-0.05983339986693...|
2020-02-01| United States|-0.10244055453991477|
2020-02-01| Singapore| -0.1093503676470588|
2020-02-01| Japan|-0.07659232613908873|
2020-02-01| Sweden|-0.07364786885245903|
+----------+--------------+--------------------+

In [0]:
df4.write.format("jdbc") \
  .mode("append") \
  .option("url", "") \
  .option("dbtable", "") \
  .option("user", "") \
  .option("password", "") \
  .save()

In [0]:
for i in range(1, 32):
  if i == 13:
    continue
  df = spark.read.csv("wasbs://mar@sentimentdatacs4225.blob.core.windows.net/mar-" + str(i) + ".csv")
  total = df.count()
#   ratio = 10000/total;
#   df = df.sample(False, ratio, 4225)
  print("Current Day" + str(i))
  print("the count for df",df.count())
  udfsomefunc = F.udf(get_sentiment, StringType())
  df3 = df.withColumn("sentiment",udfsomefunc(df._c3))
  intermeiatefilename = "wasbs://mar@sentimentdatacs4225.blob.core.windows.net/mar-sentiment-NLTK-" + str(i) + ".csv"
  df3.write.format("csv").save(intermeiatefilename)
  df3 = spark.read.csv(intermeiatefilename)
  df3=df3.withColumnRenamed("_c0","Date").withColumnRenamed("_c1","Language").withColumnRenamed("_c2","Country").withColumnRenamed("_c3","tweet").withColumnRenamed("_c4","sentimentScore")
  df4=df3.groupby("Date","Country").agg(avg("sentimentScore").alias("sentimentScore_avg"),count("sentimentScore").alias("tweetCount"))
  df4.write.format("jdbc") \
  .mode("append") \
  .option("url", "") \
  .option("dbtable", "") \
  .option("user", "") \
  .option("password", "") \
  .save()

Current Day1
the count for df 140940
Current Day2
the count for df 159011
Current Day3
the count for df 164617
Current Day4
the count for df 186024
Current Day5
the count for df 209832
Current Day6
the count for df 191051
Current Day7
the count for df 165502
Current Day8
the count for df 165099
Current Day9
the count for df 220162
Current Day10
the count for df 207830
Current Day11
the count for df 227717
Current Day12
the count for df 307047
Current Day14
the count for df 288975
Current Day15
the count for df 354261
Current Day16
the count for df 358885
Current Day17
the count for df 384033
Current Day18
the count for df 191977
Current Day19
the count for df 147531
Current Day20
the count for df 171350
Current Day21
the count for df 164829
Current Day22
the count for df 356979
Current Day23
the count for df 472172
Current Day24
the count for df 448602
Current Day25
the count for df 296700
Current Day26
the count for df 528959
Current Day27
the count for df 477831
Current Day28
the count for df 524970
Current Day29
the count for df 567585
Current Day30
the count for df 572010
Current Day31
the count for df 586031